In [1]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output, State

In [2]:
import pandas as pd

# Load data
df = pd.read_csv('stockdata2.csv', index_col=0, parse_dates=True)
df.index = pd.to_datetime(df['Date'])

In [3]:
df

,Date,stock,value,change
Date,,,,
2007-01-03,2007-01-03,MSFT,23.950705,-0.167452
2007-01-03,2007-01-03,IBM,80.517962,1.069189
2007-01-03,2007-01-03,SBUX,16.149666,0.113476
2007-01-03,2007-01-03,AAPL,11.086612,2.219569
2007-01-03,2007-01-03,GSPC,1416.599976,0.122829
...,...,...,...,...
2016-02-29,2016-02-29,MSFT,50.880001,3.341197
2016-02-29,2016-02-29,IBM,131.029999,2.549032
2016-02-29,2016-02-29,SBUX,58.209999,3.143793


In [4]:
df.tail()

,Date,stock,value,change
Date,,,,
2016-02-29,2016-02-29,MSFT,50.880001,3.341197
2016-02-29,2016-02-29,IBM,131.029999,2.549032
2016-02-29,2016-02-29,SBUX,58.209999,3.143793
2016-02-29,2016-02-29,AAPL,96.690002,3.971452
2016-02-29,2016-02-29,GSPC,1932.229980,2.386879


In [5]:
# to show the count value of stokes:
df['stock'].value_counts()

MSFT    2305
GSPC    2305
AAPL    2305
SBUX    2305
IBM     2305
Name: stock, dtype: int64

In [6]:
# Unique stokes:
df['stock'].unique()

array(['MSFT', 'IBM', 'SBUX', 'AAPL', 'GSPC'], dtype=object)

In [7]:
# Creates a list of dictionaries, which have the keys 'label' and 'value':
# which append each stoke in dictionary: 

def get_options(list_stocks):
    dict_list = []
    for i in list_stocks:
        dict_list.append({'label': i, 'value': i})

    return dict_list

get_options(df['stock'].unique())

[{'label': 'MSFT', 'value': 'MSFT'},
 {'label': 'IBM', 'value': 'IBM'},
 {'label': 'SBUX', 'value': 'SBUX'},
 {'label': 'AAPL', 'value': 'AAPL'},
 {'label': 'GSPC', 'value': 'GSPC'}]

In [8]:
# Initialise the app
app= JupyterDash(external_stylesheets =['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Define the app
app.layout = html.Div()

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [9]:
app.layout = html.Div(children=[
                      html.Div(className='row',  # Define the row element
                               children=[
                                  html.Div(className='four columns div-user-controls',
                                          children = [
                                                        html.H2('Dash - STOCK PRICES'),
                                                        html.P('''Visualising time series with Plotly - Dash'''),
                                                        html.P('''Pick one or more stocks from the dropdown below.'''),
                                                        html.Div(className='div-for-dropdown',
                                                                 children=[
                                                                     dcc.Dropdown(id='stockselector',
                                                                                  options=get_options(df['stock'].unique()),
                                                                                  multi=True,
                                                                                  value=[df['stock'].sort_values()[0]],
                                                                                  style={'backgroundColor': '#1E1E1E'},
                                                                                  className='stockselector')
                                                                          ],
                                                                  style={'color': '#1E1E1E'})
                                                     ], 
                                          ),  # Define the left element
                                   
                                  html.Div(className='eight columns div-for-charts bg-grey',
                                          children= [
                                              dcc.Graph(id='timeseries',
                                                        config={'displayModeBar': False}),
                                              dcc.Graph(id='change',
                                                        config={'displayModeBar': False})])  # Define the right element
                                  ])
])


@app.callback(Output('timeseries', 'figure'),
              [Input('stockselector', 'value')])
def update_timeseries(selected_dropdown_value):
    ''' Draw traces of the feature 'value' based one the currently selected stocks '''
    # STEP 1
    trace = []  
    df_sub = df
    # STEP 2
    # Draw and append traces for each stock
    for stock in selected_dropdown_value:   
        trace.append(go.Scatter(x=df_sub[df_sub['stock'] == stock].index,
                                 y=df_sub[df_sub['stock'] == stock]['value'],
                                 mode='lines',
                                 opacity=0.7,
                                 name=stock,
                                 textposition='bottom center'))  
    # STEP 3
    traces = [trace]
    data = [val for sublist in traces for val in sublist]
    # Define Figure
    # STEP 4
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Stock Prices', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'range': [df_sub.index.min(), df_sub.index.max()]},
              ),

              }

    return figure


@app.callback(Output('change', 'figure'),
              [Input('stockselector', 'value')])
def update_change(selected_dropdown_value):
    ''' Draw traces of the feature 'change' based one the currently selected stocks '''
    trace = []
    df_sub = df
    # Draw and append traces for each stock
    for stock in selected_dropdown_value:
        trace.append(go.Scatter(x=df_sub[df_sub['stock'] == stock].index,
                                 y=df_sub[df_sub['stock'] == stock]['change'],
                                 mode='lines',
                                 opacity=0.7,
                                 name=stock,
                                 textposition='bottom center'))
    traces = [trace]
    data = [val for sublist in traces for val in sublist]
    # Define Figure
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'t': 50},
                  height=250,
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Daily Change', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'showticklabels': False, 'range': [df_sub.index.min(), df_sub.index.max()]},
              ),
              }

    return figure


# @app.callback(
#     Output('year_slider', 'children'),
#     [Input('stockselector', 'value')])

# # @app.callback(Output('year_slider', 'value'),
# #     [Input('timeseries', 'selectedData')])

# def _update_graph(year_range):
#     df['Date'] = '{}-01-01'.format(year_range[0])